<a href="https://colab.research.google.com/github/Wn013/-tool-Download-Upload-Google-Drive-version-1.5/blob/main/Winni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Copy Folder Google Drive to Google Drive - 1TouchPro

In [1]:
#@title Input (Optimized)
from ipywidgets import widgets
from IPython.display import display # Import display explicitly

# --- Định nghĩa các Widgets ---

# Sử dụng mô tả rõ ràng hơn và có thể thêm tooltip nếu cần
dest_text = widgets.Text(
    description="Thư mục Đích:",
    placeholder='Dán link thư mục Google Drive của bạn',
    style={'description_width': 'initial'}, # Cho phép mô tả dài hơn nếu cần
    layout=widgets.Layout(width='90%') # Chiều rộng lớn hơn
)

source_text = widgets.Text(
    description="Thư mục Nguồn:",
    placeholder='Dán link thư mục Google Drive nguồn (có thể là shared)',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='90%')
)

# Sử dụng IntText cho các giá trị số nguyên như số trang
# Giá trị 0 có nghĩa là không giới hạn (hoặc từ đầu/đến cuối)
from_page_text = widgets.IntText(
    description="Từ trang (0=từ đầu):",
    value=0, # Giá trị mặc định là số 0
    style={'description_width': 'initial'}
)

to_page_text = widgets.IntText(
    description="Đến trang (0=đến cuối):",
    value=0, # Giá trị mặc định là số 0
    style={'description_width': 'initial'}
)

# Sử dụng FloatText cho dung lượng để cho phép số lẻ (ví dụ: 1.5 GB)
# Lưu ý: Code xử lý chính sau này đã đặt cứng giới hạn là 1024 GB (1TB)
# Widget này có thể không còn tác dụng nếu bạn không sửa lại phần Main.
max_download_size_text = widgets.FloatText(
    description="Tổng dung lượng tối đa(GB) (0=không giới hạn):",
    value=0, # Đặt mặc định là 0 (không giới hạn) thay vì số lớn
    style={'description_width': 'initial'}
    # Bạn có thể bỏ widget này nếu code sau luôn đặt cứng là 1TB
)

exclude_str_text = widgets.Text(
    description="Loại trừ file/folder chứa:",
    placeholder='Phân cách bởi dấu phẩy, ví dụ: .zip, Thư mục tạm',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='90%')
)


# --- Hiển thị Widgets ---

# Sử dụng VBox để sắp xếp các widget theo chiều dọc cho gọn gàng
print("Vui lòng nhập thông tin:")
input_widgets = widgets.VBox([
    dest_text,
    source_text,
    from_page_text,
    to_page_text,
    max_download_size_text, # Cân nhắc ẩn hoặc xóa nếu không dùng
    exclude_str_text
])

display(input_widgets)

Vui lòng nhập thông tin:


In [ ]:
# @title Run (Optimized Version)
import os
import time
import re
import sys
import typing as t  # Thêm thư viện typing để sử dụng type hints
import math       # Thêm math để sử dụng cho exponential backoff
from googleapiclient.discovery import build, Resource # Import Resource để type hint service
# Import HttpError để xử lý lỗi API cụ thể
from googleapiclient.errors import HttpError
from google.colab import auth
from google.colab import drive # Mặc dù import nhưng không thấy sử dụng trực tiếp, có thể để lại
from tqdm.notebook import tqdm # Thêm tqdm cho progress bar trong Colab

# --- Constants ---
FOLDER_MIME_TYPE = 'application/vnd.google-apps.folder'
MAX_RETRIES = 5 # Số lần thử lại tối đa cho API call
INITIAL_RETRY_DELAY = 1.0 # Thời gian chờ ban đầu (giây)

class DownloadFromDrive:
    # Sử dụng type hints để rõ ràng hơn về kiểu dữ liệu
    def __init__(self):
        self._total_size: float = 0.0 # Tổng kích thước đã copy (MB)
        self._limit_size: float = 0.0 # Giới hạn kích thước (GB) - sẽ được đặt ở phần Main
        self.excluded_strings: t.List[str] = []

    def get_user_credential(self) -> t.Optional[Resource]:
        """Xác thực người dùng và trả về đối tượng Drive service."""
        try:
            auth.authenticate_user()
            # Sử dụng discovery cache để tránh cảnh báo, trả về Resource hoặc None nếu lỗi
            drive_service: Resource = build('drive', 'v3', cache_discovery=False)
            return drive_service
        except Exception as e:
            print(f"Lỗi khi xác thực hoặc tạo Drive service: {e}")
            return None

    def get_childs_from_folder(self, drive_service: Resource, folder_id: str, from_page: int, to_page: int) -> t.List[t.Dict[str, t.Any]]:
        """Lấy danh sách các mục con từ một thư mục, hỗ trợ phân trang và loại trừ."""
        files: t.List[t.Dict[str, t.Any]] = []
        page_token: t.Optional[str] = None
        # Query cơ bản
        query = f"'{folder_id}' in parents and trashed = false"
        # Thêm điều kiện loại trừ nếu có
        if self.excluded_strings:
            # Dùng "and not" thay vì "and (not ...)" để đơn giản hơn nếu chỉ có 1 điều kiện
            # Nếu nhiều, cần dùng join như cũ
            not_contains_query = " and ".join([f"not name contains '{ext}'" for ext in self.excluded_strings])
            query += f" and {not_contains_query}" # Bỏ dấu ngoặc đơn nếu logic phù hợp

        pages = 0
        print(f"Fetching file list from folder ID: {folder_id}...")
        while True:
            try:
                pages += 1
                # print(f"Requesting page {pages}...") # Bật nếu cần debug pagination
                response = drive_service.files().list(
                    q=query,
                    orderBy='name, createdTime', # Sắp xếp để thứ tự ổn định
                    fields='files(id, name, mimeType, size), nextPageToken', # Chỉ lấy trường cần thiết
                    pageToken=page_token,
                    supportsAllDrives=True,
                    includeItemsFromAllDrives=True,
                    pageSize=1000 # Lấy nhiều kết quả mỗi lần gọi
                ).execute()

                current_files = response.get('files', [])
                print(f"Page {pages}: Found {len(current_files)} items.")
                # Chỉ thêm vào danh sách nếu nằm trong khoảng trang yêu cầu (hoặc to_page=0 là lấy hết)
                if to_page == 0 or (from_page <= pages <= to_page):
                    files.extend(current_files)

                page_token = response.get('nextPageToken', None)
                # Dừng nếu hết token hoặc đã đạt trang cuối cùng yêu cầu (và to_page > 0)
                if page_token is None or (to_page > 0 and pages >= to_page):
                    print("Reached end of file list or target page.")
                    break

            except HttpError as error:
                print(f"An HTTP error occurred while listing files (Page {pages}): {error}")
                # Có thể thêm retry logic ở đây nếu muốn
                break # Thoát vòng lặp nếu có lỗi khi list file
            except Exception as e:
                print(f"An unexpected error occurred while listing files (Page {pages}): {str(e)}")
                break # Thoát vòng lặp

        print(f"Total files/folders fetched in specified range: {len(files)}")
        return files

    # --- HÀM SAO CHÉP VỚI RETRY LOGIC ---
    def _execute_with_retry(self, api_call: t.Callable[[], t.Any]) -> t.Optional[t.Dict[str, t.Any]]:
        """Thực thi một API call với cơ chế retry (exponential backoff)."""
        retry_delay = INITIAL_RETRY_DELAY
        for attempt in range(MAX_RETRIES):
            try:
                return api_call() # Thực thi hàm API được truyền vào
            except HttpError as error:
                # Chỉ thử lại với các lỗi tạm thời (Rate Limit, Server Error)
                if error.resp.status in [403, 429, 500, 502, 503, 504]:
                    print(f"--- API Error ({error.resp.status}) on attempt {attempt + 1}/{MAX_RETRIES}. Retrying in {retry_delay:.2f}s...")
                    time.sleep(retry_delay)
                    # Exponential backoff: nhân đôi thời gian chờ, thêm chút ngẫu nhiên
                    retry_delay = min(retry_delay * 2 + (random.random() * 0.5), 60) # Giới hạn max delay là 60s
                else:
                    # Lỗi không thể retry (404, 401, 400, ...) -> báo lỗi và không thử lại
                    print(f"--- Unrecoverable HTTP error occurred: {error}")
                    return None # Trả về None để hàm gọi biết là lỗi không thử lại được
            except Exception as e:
                # Các lỗi không phải HTTP
                print(f"--- An unexpected error occurred during API call: {e}")
                return None # Không thử lại lỗi không xác định
        print(f"--- API call failed after {MAX_RETRIES} attempts.")
        return None

    # ==============================================================
    # HÀM COPY_FILE ĐÃ ĐƯỢC SỬA ĐỔI VỚI RETRY VÀ CHECK TỒN TẠI CHÍNH XÁC HƠN
    # ==============================================================
    def copy_file(self, drive_service: Resource, dest_folder_id: str, source_file: t.Dict[str, t.Any]):
        """Sao chép một tệp hoặc xử lý thư mục con (đệ quy), có retry và kiểm tra tồn tại."""
        file_name = source_file.get('name', 'Unknown File')
        file_id = source_file.get('id')
        file_mime_type = source_file.get('mimeType')

        if not file_id:
            print(f"--- Skipping item [{file_name}] - Missing file ID.")
            return

        # --- Xử lý trường hợp là FILE ---
        if file_mime_type != FOLDER_MIME_TYPE:
            # Kiểm tra file tồn tại chính xác bằng tên và mimeType
            if not self.check_if_exists(drive_service, dest_folder_id, file_name, file_mime_type):
                print(f"Attempting to copy: [{file_name}] (ID: {file_id})")
                body_file_inf = {'parents': [dest_folder_id], 'name': file_name} # Giữ nguyên tên khi copy

                # Định nghĩa API call để truyền vào hàm retry
                def copy_api_call():
                    return drive_service.files().copy(
                        body=body_file_inf,
                        fileId=file_id,
                        supportsAllDrives=True,
                        fields='id, name, size' # Lấy các trường cần thiết
                    ).execute()

                start_time = time.time()
                # Thực thi copy với retry
                copied_file_info = self._execute_with_retry(copy_api_call)
                end_time = time.time()

                # Nếu copy thành công (không phải None)
                if copied_file_info:
                    # Ưu tiên size từ file gốc vì API copy đôi khi trả về size=0 ban đầu
                    filesize = int(source_file.get('size', 0)) # Lấy size gốc nếu có
                    if filesize == 0 and copied_file_info.get('size'): # Nếu gốc = 0, thử lấy size từ kết quả copy
                       filesize = int(copied_file_info.get('size', 0))

                    if filesize > 0:
                        size_mb = filesize / (1024 * 1024)
                        self._total_size += size_mb
                        duration = end_time - start_time
                        speed_mb = size_mb / duration if duration > 0.001 else 0
                        print(f"[{file_name}] copied successfully. Size [{size_mb:0.2f}] MB. Speed [{speed_mb:0.2f}] MB/s")
                    else:
                        print(f"[{file_name}] copied (size 0 or unknown).")

                    # Kiểm tra giới hạn dung lượng
                    if self._limit_size > 0 and self._total_size >= (self._limit_size * 1024):
                        self.on_total_size_exceeded(f"Total size {self._total_size:.2f} MB exceeds limit {self._limit_size} GB. Ending the program.")
                else:
                    # Nếu _execute_with_retry trả về None, nghĩa là đã retry hết cách hoặc gặp lỗi không thể retry
                    print(f"--- Failed to copy file [{file_name}] (ID: {file_id}) after retries or due to unrecoverable error.")
            else:
                # File đã tồn tại
                print(f"[{file_name}] exists. Skipping.")

        # --- Xử lý trường hợp là THƯ MỤC ---
        else:
            print(f"\nProcessing folder: [{file_name}] (ID: {file_id})")
            # Kiểm tra xem thư mục đích đã tồn tại chưa
            existing_sub_folder_id = self.check_if_exists(drive_service, dest_folder_id, file_name, FOLDER_MIME_TYPE)
            sub_folder_id = existing_sub_folder_id

            if not existing_sub_folder_id:
                # Nếu chưa tồn tại, tạo mới (với retry)
                print(f"Creating subfolder [{file_name}] in destination.")
                sub_folder_id = self.create_folder(drive_service, dest_folder_id, file_name) # create_folder đã có retry

            if sub_folder_id: # Chỉ tiếp tục nếu tạo hoặc tìm thấy thư mục đích thành công
                print(f"Entering folder [{file_name}] (Destination ID: {sub_folder_id})...")
                # Lấy danh sách con từ thư mục nguồn (chỉ lấy trang 0 - tức là tất cả con trực tiếp)
                source_files_in_folder = self.get_childs_from_folder(drive_service, file_id, 0, 0)
                if source_files_in_folder:
                    print(f"Found {len(source_files_in_folder)} items inside [{file_name}]. Starting copy...")
                    # Gọi đệ quy để sao chép nội dung bên trong
                    self.copy_multiple_files(drive_service, sub_folder_id, source_files_in_folder)
                    print(f"Finished copying contents of folder [{file_name}].\n")
                else:
                     print(f"Source folder [{file_name}] is empty. Nothing to copy inside.")
            else:
                # Lỗi khi tạo/tìm thư mục đích
                print(f"Failed to create or find destination subfolder [{file_name}]. Skipping contents.")

    # ==============================================================
    # KẾT THÚC HÀM COPY_FILE ĐÃ SỬA
    # ==============================================================

    def create_folder(self, drive_service: Resource, dest_folder_id: str, sub_folder_name: str) -> t.Optional[str]:
        """Tạo thư mục, kiểm tra tồn tại trước, có retry."""
        # Kiểm tra tồn tại chính xác bằng tên và mimeType
        exist_folder_id = self.check_if_exists(drive_service, dest_folder_id, sub_folder_name, FOLDER_MIME_TYPE)
        if exist_folder_id:
             print(f"Folder '{sub_folder_name}' already exists with ID: {exist_folder_id}. Using existing.")
             return exist_folder_id
        else:
            print(f"Attempting to create folder: {sub_folder_name}")
            sub_folder_inf = {'name': sub_folder_name, 'mimeType': FOLDER_MIME_TYPE, 'parents': [dest_folder_id]}

            # Định nghĩa API call để retry
            def create_api_call():
                return drive_service.files().create(
                    body=sub_folder_inf,
                    fields='id',
                    supportsAllDrives=True
                ).execute()

            # Thực thi tạo với retry
            folder_info = self._execute_with_retry(create_api_call)

            if folder_info and folder_info.get('id'):
                folder_id = folder_info.get('id')
                print(f"Folder '{sub_folder_name}' created successfully with ID: {folder_id}")
                return folder_id
            else:
                print(f"--- Failed to create folder [{sub_folder_name}] after retries or due to unrecoverable error.")
                return None

    # --- CẬP NHẬT check_if_exists ---
    def check_if_exists(self, drive_service: Resource, dest_folder_id: str, name: str, mime_type: t.Optional[str] = None) -> t.Optional[str]:
        """Kiểm tra sự tồn tại của tệp/thư mục với tên chính xác và tùy chọn mimeType."""
        try:
            # Thoát ký tự ' và \ trong tên file cho truy vấn q
            processed_name = name.replace("\\", "\\\\").replace("'", "\\'")
            # --- SỬ DỤNG name = thay vì name contains ---
            query = f"'{dest_folder_id}' in parents and name = '{processed_name}' and trashed=false"
            # Thêm mimeType vào query nếu được cung cấp
            if mime_type:
                query += f" and mimeType = '{mime_type}'"

            # Không cần retry cho list vì thường ít lỗi hơn và chỉ cần biết có hay không
            results = drive_service.files().list(
                q=query,
                fields='files(id, name)', # Lấy thêm name để debug nếu cần
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
                pageSize=1 # Chỉ cần tìm 1 là đủ
            ).execute()

            files = results.get('files', [])
            if files:
                # print(f"Found existing item: Name='{files[0]['name']}', ID='{files[0]['id']}' for query name='{name}'")
                return files[0]['id'] # Trả về ID của mục đầu tiên tìm thấy

        except HttpError as error:
            # Lỗi 404 không phải là lỗi ở đây, chỉ là không tìm thấy
            if error.resp.status != 404:
                print(f"An HTTP error occurred while checking existence for '{name}': {error}")
        except Exception as e:
            print(f"An unexpected error occurred while checking existence for '{name}': {e}")

        return None # Trả về None nếu không tìm thấy hoặc có lỗi

    # --- CẬP NHẬT copy_multiple_files với TQDM ---
    def copy_multiple_files(self, drive_service: Resource, dest_folder_id: str, source_files: t.List[t.Dict[str, t.Any]]):
        """Sao chép nhiều tệp/thư mục với thanh tiến trình tqdm."""
        total_files = len(source_files)
        if total_files == 0:
            print("No items to copy in this batch.")
            return

        # Sử dụng tqdm để tạo progress bar
        # desc: Mô tả hiển thị trên thanh tiến trình
        # unit: Đơn vị của mỗi bước (ở đây là "item")
        # leave=False: Xóa thanh tiến trình khi hoàn thành (tùy chọn)
        for source_file in tqdm(source_files, desc=f"Copying to {dest_folder_id[:10]}...", unit="item", leave=True):
            self.copy_file(drive_service, dest_folder_id, source_file)
            # Có thể giữ lại sleep nhỏ nếu vẫn gặp rate limit dù đã có retry
            # time.sleep(0.05)

    def extract_folder_id_from_url(self, url: str) -> t.Optional[str]:
        """Trích xuất ID thư mục/tệp từ URL Google Drive."""
        patterns = [
            r'/folders/([-\w]{25,})',
            r'/drive/folders/([-\w]{25,})',
            r'id=([-\w]{25,})',
            r'/file/d/([-\w]{25,})'
        ]
        for pattern in patterns:
            match = re.search(pattern, url)
            if match:
                return match.group(1)
        # Thử lấy phần cuối nếu không khớp pattern
        if '/' in url:
            potential_id = url.split('?')[0].split('/')[-1] # Lấy phần cuối trước dấu ?
            if len(potential_id) >= 25 and not '.' in potential_id and potential_id != 'edit':
                 return potential_id
        print(f"Warning: Could not automatically extract ID from URL: {url}")
        return None

    def on_total_size_exceeded(self, message: str):
        """Hàm xử lý khi vượt quá giới hạn dung lượng."""
        print(f"\n!!! LIMIT EXCEEDED !!!")
        print(message)
        print(f"Total size copied: {self._total_size / 1024:.2f} GB")
        sys.exit(1) # Thoát chương trình với mã lỗi

    def copy_drive_to_drive(self, destDriveLink: str, sourceDriveLink: str, from_page: int, to_page: int):
        """Hàm chính điều phối quá trình sao chép."""
        print("Starting Drive to Drive copy process...")
        service = self.get_user_credential()
        if not service:
            print("Failed to get Google Drive credentials. Aborting.")
            return

        start_time_total = time.time()
        dest_folder_id_parent = self.extract_folder_id_from_url(destDriveLink)
        source_folder_id = self.extract_folder_id_from_url(sourceDriveLink)

        if not dest_folder_id_parent:
            print(f"Error: Could not extract destination folder ID from URL: {destDriveLink}")
            return
        if not source_folder_id:
            print(f"Error: Could not extract source folder ID from URL: {sourceDriveLink}")
            return

        print(f"Source Folder ID: {source_folder_id}")
        print(f"Destination Parent Folder ID: {dest_folder_id_parent}")
        # In giới hạn thực tế đã được đặt
        if self._limit_size > 0:
             print(f"Effective Size Limit: {self._limit_size} GB")
        else:
             print("Effective Size Limit: Unlimited")
        if self.excluded_strings:
             print(f"Excluding files containing: {self.excluded_strings}")


        try:
            # Lấy thông tin thư mục nguồn để lấy tên
            print(f"Fetching source folder info...")
            # Dùng retry cho cả get thông tin folder nguồn
            def get_source_folder_call():
                 return service.files().get(fileId=source_folder_id, supportsAllDrives=True, fields='id, name').execute()
            source_folder = self._execute_with_retry(get_source_folder_call)

            if not source_folder:
                 print(f"Error: Could not fetch source folder information (ID: {source_folder_id}). Check ID and permissions. Aborting.")
                 return

            source_folder_name = source_folder.get('name', 'Copied_Folder') # Đặt tên mặc định nếu không lấy được
            print(f"Source folder name: '{source_folder_name}'")

            # Tạo thư mục đích chính (tên giống thư mục nguồn) bên trong destDriveLink (với retry)
            print(f"Ensuring main destination folder '{source_folder_name}' exists inside {dest_folder_id_parent}...")
            # create_folder đã bao gồm check_if_exists và retry
            new_dest_folder_id = self.create_folder(service, dest_folder_id_parent, source_folder_name)

            if not new_dest_folder_id:
                print(f"Error: Failed to create/find main destination folder '{source_folder_name}'. Aborting.")
                return

            print(f"Destination folder ready: '{source_folder_name}' (ID: {new_dest_folder_id})")

            # Lấy danh sách file/folder con từ thư mục nguồn theo trang
            print(f"Fetching item list from source folder '{source_folder_name}'...")
            source_files = self.get_childs_from_folder(service, source_folder_id, from_page, to_page)

            # Bắt đầu sao chép hàng loạt với progress bar
            if source_files:
                self.copy_multiple_files(service, new_dest_folder_id, source_files)
            else:
                print("No files or folders found in the source directory (within specified page range) to copy.")

        except HttpError as error:
            print(f"\nAn critical HTTP error occurred during the process: {error}")
            if error.resp.status == 404:
                print("Please double-check if the Source or Destination Folder IDs are correct and you have access permissions.")
        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            import traceback
            traceback.print_exc() # In traceback để debug lỗi lạ


        end_time_total = time.time()
        total_duration_seconds = end_time_total - start_time_total
        total_duration_minutes = total_duration_seconds / 60

        # Tính toán cuối cùng
        size_gb = self._total_size / 1024
        speed_mb_avg = self._total_size / total_duration_seconds if total_duration_seconds > 0.01 else 0

        print("\n================== SUMMARY ==================")
        print(f"Copy process finished.")
        print(f"Total Size Copied: {size_gb:.3f} GB ({self._total_size:.2f} MB)")
        print(f"Total Time: {total_duration_seconds:.2f} seconds ({total_duration_minutes:.2f} minutes)")
        print(f"Average Speed: {speed_mb_avg:.2f} MB/s (based on tracked file sizes)")
        print(f"Destination: Folder '{source_folder_name}' inside parent ID {dest_folder_id_parent}")
        print("===========================================")


# ==============================================================
# PHẦN MAIN ĐỂ CHẠY TRONG COLAB - CẬP NHẬT GIỚI HẠN VÀ KHỞI TẠO
# ==============================================================
# Thêm thư viện random để dùng trong exponential backoff
import random

# --- Giả sử các widgets đã được định nghĩa trong ô Colab trước ---
# dest_text = widgets.Text(description="Dest Link:")
# source_text = widgets.Text(description="Source Link:")
# from_page_text = widgets.Text(description="From Page:", value="0")
# to_page_text = widgets.Text(description="To Page:", value="0")
# max_download_size_text = widgets.Text(description="Max Size(GB):", value="0") # Mặc định 0 = không giới hạn
# exclude_str_text = widgets.Text(description="Exclude Ext (csv):")

# --- Lấy giá trị từ widgets một cách an toàn ---
destDriveLink = ""
sourceDriveLink = ""
fromPage = 0
toPage = 0
limit_gb_input = 0.0 # Mặc định không giới hạn
exclude_input = ""

# Sử dụng locals() để kiểm tra sự tồn tại của biến widget trước khi truy cập .value
if 'dest_text' in locals() and hasattr(locals()['dest_text'], 'value'):
    destDriveLink = locals()['dest_text'].value
if 'source_text' in locals() and hasattr(locals()['source_text'], 'value'):
    sourceDriveLink = locals()['source_text'].value
if 'from_page_text' in locals() and hasattr(locals()['from_page_text'], 'value'):
    try: fromPage = int(locals()['from_page_text'].value)
    except (ValueError, TypeError): fromPage = 0
if 'to_page_text' in locals() and hasattr(locals()['to_page_text'], 'value'):
    try: toPage = int(locals()['to_page_text'].value)
    except (ValueError, TypeError): toPage = 0
if 'exclude_str_text' in locals() and hasattr(locals()['exclude_str_text'], 'value'):
    exclude_input = locals()['exclude_str_text'].value

# --- Lấy và xác thực giới hạn dung lượng từ widget ---
if 'max_download_size_text' in locals() and hasattr(locals()['max_download_size_text'], 'value'):
    try:
        limit_gb_input = float(locals()['max_download_size_text'].value)
        if limit_gb_input < 0:
            print("Warning: Negative size limit entered. Setting limit to 0 (Unlimited).")
            limit_gb_input = 0
    except (ValueError, TypeError):
        print("Warning: Invalid size limit input. Setting limit to 0 (Unlimited).")
        limit_gb_input = 0
else:
     print("Info: Size limit widget not found. Setting limit to 0 (Unlimited).")
     limit_gb_input = 0 # Mặc định không giới hạn nếu widget không tồn tại


# --- TẠO INSTANCE VÀ CHẠY ---
downloader = DownloadFromDrive()

# --- Đặt các tham số cho instance ---
downloader._limit_size = limit_gb_input # Sử dụng giá trị đã xác thực từ widget

# Xử lý chuỗi loại trừ
downloader.excluded_strings = [ext.strip() for ext in exclude_input.split(",") if ext.strip()]


# --- Kiểm tra link và chạy ---
if not destDriveLink or not sourceDriveLink:
    print("\n" + "="*40)
    print("!!! ERROR: Please provide valid Source and Destination Google Drive links in the widgets above. !!!")
    print("Example Source Link: https://drive.google.com/drive/folders/YOUR_SOURCE_FOLDER_ID")
    print("Example Dest Link:   https://drive.google.com/drive/folders/YOUR_DESTINATION_PARENT_FOLDER_ID")
    print("="*40 + "\n")
else:
    # Chạy chức năng chính
    downloader.copy_drive_to_drive(destDriveLink, sourceDriveLink, fromPage, toPage)

Starting Drive to Drive copy process...
Source Folder ID: 1ns28QB_WLpWujoXWVs6Bm0qsmRLIj3cN
Destination Parent Folder ID: 1bcF1cO4FKRH2PeNFeOWTF2jaerHD2oPc
Effective Size Limit: Unlimited
Fetching source folder info...
Source folder name: 'TỔNG HỢP - GIÁO TRÌNH SV - TÀI LIỆU KHÁC'
Ensuring main destination folder 'TỔNG HỢP - GIÁO TRÌNH SV - TÀI LIỆU KHÁC' exists inside 1bcF1cO4FKRH2PeNFeOWTF2jaerHD2oPc...
Attempting to create folder: TỔNG HỢP - GIÁO TRÌNH SV - TÀI LIỆU KHÁC
Folder 'TỔNG HỢP - GIÁO TRÌNH SV - TÀI LIỆU KHÁC' created successfully with ID: 1NMhTHGHxR4eaHtTdoLffiumhWj03aOsJ
Destination folder ready: 'TỔNG HỢP - GIÁO TRÌNH SV - TÀI LIỆU KHÁC' (ID: 1NMhTHGHxR4eaHtTdoLffiumhWj03aOsJ)
Fetching item list from source folder 'TỔNG HỢP - GIÁO TRÌNH SV - TÀI LIỆU KHÁC'...
Fetching file list from folder ID: 1ns28QB_WLpWujoXWVs6Bm0qsmRLIj3cN...
Page 1: Found 9 items.
Reached end of file list or target page.
Total files/folders fetched in specified range: 9


Copying to 1NMhTHGHxR...:   0%|          | 0/9 [00:00<?, ?item/s]


Processing folder: [(0) TỔNG HỢP GIÁO TRÌNH ĐẠI HỌC TAILIEUONTHI.NET] (ID: 1RVGSzHCfOesJGtpcra84oYH7l2LdI1ag)
Creating subfolder [(0) TỔNG HỢP GIÁO TRÌNH ĐẠI HỌC TAILIEUONTHI.NET] in destination.
Attempting to create folder: (0) TỔNG HỢP GIÁO TRÌNH ĐẠI HỌC TAILIEUONTHI.NET
Folder '(0) TỔNG HỢP GIÁO TRÌNH ĐẠI HỌC TAILIEUONTHI.NET' created successfully with ID: 1PQXZ61HXY-7mhChgEjNdskQ3B9v-jOAw
Entering folder [(0) TỔNG HỢP GIÁO TRÌNH ĐẠI HỌC TAILIEUONTHI.NET] (Destination ID: 1PQXZ61HXY-7mhChgEjNdskQ3B9v-jOAw)...
Fetching file list from folder ID: 1RVGSzHCfOesJGtpcra84oYH7l2LdI1ag...
Page 1: Found 0 items.
Reached end of file list or target page.
Total files/folders fetched in specified range: 0
Source folder [(0) TỔNG HỢP GIÁO TRÌNH ĐẠI HỌC TAILIEUONTHI.NET] is empty. Nothing to copy inside.

Processing folder: [BÀI GIẢNG VÀ BÀI TẬP VẬT LÝ ĐẠI CƯƠNG 1] (ID: 1h9l9B-KAZvki1QreRSJo1UhVdgGoRziv)
Creating subfolder [BÀI GIẢNG VÀ BÀI TẬP VẬT LÝ ĐẠI CƯƠNG 1] in destination.
Attempting to cre

Copying to 1gFHLhmaRr...:   0%|          | 0/26 [00:00<?, ?item/s]

Attempting to copy: [Bai tap. Chuong 1.1. Dong hoc chat diem-gui len mang.pdf] (ID: 1KOEGb8lF89_dFGZyh-zrRe-7ooy_FbW1)
[Bai tap. Chuong 1.1. Dong hoc chat diem-gui len mang.pdf] copied successfully. Size [0.16] MB. Speed [0.08] MB/s
Attempting to copy: [Bai tap. Chuong 1.2. Dong luc hoc chat diem - gui len mang.pdf] (ID: 1vsS6bV8FFPoAcYv3GQtfJgmgPL7tSVke)
[Bai tap. Chuong 1.2. Dong luc hoc chat diem - gui len mang.pdf] copied successfully. Size [0.20] MB. Speed [0.13] MB/s
Attempting to copy: [Bai tap. Chuong 12. Dao dong-gui len mang.pdf] (ID: 1hbl0bJ4X5jDLS8gJna_3fNbRlKk6dw5e)
[Bai tap. Chuong 12. Dao dong-gui len mang.pdf] copied successfully. Size [0.21] MB. Speed [0.13] MB/s
Attempting to copy: [Bai tap. Chuong 13. Song co - song dien tu - gui len mang.pdf] (ID: 1lTtvgYcWUdS7mvXfuMQp1OlV0h-3mHFC)
[Bai tap. Chuong 13. Song co - song dien tu - gui len mang.pdf] copied successfully. Size [0.18] MB. Speed [0.10] MB/s
Attempting to copy: [Bai tap. Chuong 2. Co hoc vat ran-gui len mang.

...

Attempting to copy: [BKeL_Chao mung_SV_K2012.doc] (ID: 1Lrhp1qhoO89lwqaXuf4fw2p55bpMKh6r)
[BKeL_Chao mung_SV_K2012.doc] copied successfully. Size [0.02] MB. Speed [0.02] MB/s
Attempting to copy: [DHBK_Chaomung_SV.doc] (ID: 1s7RM_Fwt2XcRkuA4Ppm5eBwcSkY3-e-R)
[DHBK_Chaomung_SV.doc] copied successfully. Size [0.03] MB. Speed [0.02] MB/s
Attempting to copy: [DHBK_Tb_Dichvu_BKDT.doc] (ID: 1O0wzTENpykfNOMBREZti94mwqaJcRxaX)
[DHBK_Tb_Dichvu_BKDT.doc] copied successfully. Size [0.05] MB. Speed [0.01] MB/s
Attempting to copy: [lich SHCD dau khoa - chinh quy a3.doc] (ID: 1L-jOjmoFVU5UvlXzjjgHnBdmqwkNDnYK)
[lich SHCD dau khoa - chinh quy a3.doc] copied successfully. Size [0.08] MB. Speed [0.04] MB/s
Attempting to copy: [lich SHCD dot 1 (trang thong bao).doc] (ID: 1ugilwk0j8zltOncju_mG7tkIgtbmReGD)
[lich SHCD dot 1 (trang thong bao).doc] copied successfully. Size [0.04] MB. Speed [0.03] MB/s
Finished copying contents of folder [Thong tin chao mung].

Finished copying contents of folder [Hoc tap].


Copying to 1xzrJU9rfs...:   0%|          | 0/3 [00:00<?, ?item/s]

Attempting to copy: [de_on_2012_1_2.pdf] (ID: 1bNrqLn2bG5qdAkexx5mopf8zzyutmtIV)
[de_on_2012_1_2.pdf] copied successfully. Size [0.10] MB. Speed [0.06] MB/s
Attempting to copy: [HUONG_DAN_GIAI_CAC_PHUONG_TRINH_VI_PHAN_CAP_1.doc] (ID: 1mSe7xbGRQ3lVaggiO7WSY12oHZiF046I)
[HUONG_DAN_GIAI_CAC_PHUONG_TRINH_VI_PHAN_CAP_1.doc] copied successfully. Size [0.04] MB. Speed [0.02] MB/s
Attempting to copy: [in.docx] (ID: 1em8DQy8SdcSAZgWfx6aAv7BjRw_YLqnu)
[in.docx] copied successfully. Size [0.03] MB. Speed [0.01] MB/s
Finished copying contents of folder [In tai lieu].

Attempting to copy: [KT Ly.doc] (ID: 1r-aPrfEC5veCJW6rDzluD1__KMa9m3c3)
[KT Ly.doc] copied successfully. Size [2.92] MB. Speed [1.67] MB/s
Attempting to copy: [Lich thi.docx] (ID: 1tufqeOCI0ivDos-lWOKsuKRQ8HPBLE6Z)
[Lich thi.docx] copied successfully. Size [0.01] MB. Speed [0.01] MB/s

Processing folder: [Listen In 2] (ID: 1cwExoMxag59_W9FOkr0UZpM1MLMQXpK9)
Creating subfolder [Listen In 2] in destination.
Attempting to create folder:

Copying to 1prVvNNp7A...:   0%|          | 0/41 [00:00<?, ?item/s]

Attempting to copy: [01 01 トラック 1.wma] (ID: 1BkAco77gPu1XItChcq_yaUn9lgKP-JdY)
[01 01 トラック 1.wma] copied successfully. Size [0.40] MB. Speed [0.28] MB/s
Attempting to copy: [02 02 トラック 2.wma] (ID: 18p2IM_8wU7BFdiR1e19CZLo1AKjlRihT)
[02 02 トラック 2.wma] copied successfully. Size [1.31] MB. Speed [1.38] MB/s
Attempting to copy: [03 03 トラック 3.wma] (ID: 1yCgeUYN4Lg-uqPD4QPNjkkuPILZA5NWK)
[03 03 トラック 3.wma] copied successfully. Size [1.50] MB. Speed [1.01] MB/s
Attempting to copy: [04 04 トラック 4.wma] (ID: 1tjMNT6OawDnjW72Q-NON7xG0i-XF1Rsn)
[04 04 トラック 4.wma] copied successfully. Size [1.33] MB. Speed [1.32] MB/s
Attempting to copy: [05 05 トラック 5.wma] (ID: 1T9wnTAiUSCwXicQfN2k0a1YAFCTmBcrY)
[05 05 トラック 5.wma] copied successfully. Size [0.99] MB. Speed [0.28] MB/s
Attempting to copy: [06 06 トラック 6.wma] (ID: 16iNyg3q0WBczYm-6TFls4LJ6oFSY5Aek)
[06 06 トラック 6.wma] copied successfully. Size [1.59] MB. Speed [1.75] MB/s
Attempting to copy: [07 07 トラック 7.wma] (ID: 17lGa7nS2omr0h3qzlV4pKmEYO-l1t_zK)
[07

# Mục mới